In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!nvidia-smi

Thu Apr 10 16:09:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       1MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [3]:
# !pip install  dgl==2.0.0 -f https://data.dgl.ai/wheels/repo.html
# !pip install  dglgo==0.0.2 -f https://data.dgl.ai/wheels-test/repo.html

!pip install  dgl -f https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html

!pip install ogb

Looking in links: https://data.dgl.ai/wheels/torch-2.4/cu124/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 72.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 958.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import time

import dgl
import dgl.nn.pytorch as dglnn

import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tqdm
from ogb.nodeproppred import DglNodePropPredDataset

import gc

# Загрузка датасета

In [6]:
device = th.device("cuda")
device

device(type='cuda')

In [7]:
data = DglNodePropPredDataset(name="ogbn-products")
splitted_idx = data.get_idx_split()
train_idx, val_idx, test_idx = (
    splitted_idx["train"],
    splitted_idx["valid"],
    splitted_idx["test"],
)
graph, labels = data[0]
nfeat = graph.ndata.pop("feat").to(device)
labels = labels[:, 0].to(device)

in_feats = nfeat.shape[1]
n_classes = (labels.max() + 1).item()
# Create csr/coo/csc formats before launching sampling processes
# This avoids creating certain formats in each data loader process, which saves momory and CPU.
graph.create_formats_()
# Pack data
data = (
    train_idx,
    val_idx,
    test_idx,
    in_feats,
    labels,
    n_classes,
    nfeat,
    graph,
)

This will download 1.38GB. Will you proceed? (y/N)
 y


Downloaded 1.38 GB: 100%|██████████| 1414/1414 [00:37<00:00, 38.21it/s]


Extracting dataset/products.zip
Loading necessary files...
This might take a while.
Processing graphs...


100%|██████████| 1/1 [00:00<00:00,  1.00it/s]


Converting graphs into DGL objects...


100%|██████████| 1/1 [00:00<00:00,  2.87it/s]


Saving...


# Model

In [11]:
BATCH_SIZE = 1000
NUM_WORKERS = 4
NUM_HIDDEN = 256
NUM_LAYERS = 3
DROPOUT = 0.5
NUM_EPOCHS = 10
EVAL_EVERY = 1
LOG_EVERY = 20
SAVE_PRED = ""
FAN_OUT = "5,10,15"
LR = 0.003
WD = 0

In [14]:
SAGE(
    in_feats,
    n_classes,
    BATCH_SIZE,
    NUM_WORKERS
)

SAGE(
  (layers): ModuleList(
    (0): SAGEConv(
      (feat_drop): Dropout(p=0.0, inplace=False)
      (fc_neigh): Linear(in_features=100, out_features=256, bias=False)
      (fc_self): Linear(in_features=100, out_features=256, bias=True)
    )
    (1): SAGEConv(
      (feat_drop): Dropout(p=0.0, inplace=False)
      (fc_neigh): Linear(in_features=256, out_features=256, bias=False)
      (fc_self): Linear(in_features=256, out_features=256, bias=True)
    )
    (2): SAGEConv(
      (feat_drop): Dropout(p=0.0, inplace=False)
      (fc_neigh): Linear(in_features=256, out_features=47, bias=False)
      (fc_self): Linear(in_features=256, out_features=47, bias=True)
    )
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [8]:
class SAGE(nn.Module):
    def __init__(
        self, in_feats, n_classes, batch_size, num_workers
    ):
        super().__init__()
        self.n_layers = 3
        self.n_hidden = 256
        self.n_classes = n_classes
        self.layers = nn.ModuleList()
        self.layers.append(dglnn.SAGEConv(in_feats, self.n_hidden, "mean"))
        for i in range(1, self.n_layers - 1):
            self.layers.append(dglnn.SAGEConv(self.n_hidden, self.n_hidden, "mean"))
        self.layers.append(dglnn.SAGEConv(self.n_hidden, n_classes, "mean"))
        self.dropout = nn.Dropout(0.5)
        self.activation = F.relu
        self.batch_size = batch_size
        self.num_workers = num_workers

    def forward(self, blocks, x):
        h = x
        for l, (layer, block) in enumerate(zip(self.layers, blocks)):
            # We need to first copy the representation of nodes on the RHS from the
            # appropriate nodes on the LHS.
            # Note that the shape of h is (num_nodes_LHS, D) and the shape of h_dst
            # would be (num_nodes_RHS, D)
            h_dst = h[: block.num_dst_nodes()]
            # Then we compute the updated representation on the RHS.
            # The shape of h now becomes (num_nodes_RHS, D)
            h = layer(block, (h, h_dst))
            if l != len(self.layers) - 1:
                h = self.activation(h)
                h = self.dropout(h)
        return h

    def inference(self, g, x, device):
        """
        Inference with the GraphSAGE model on full neighbors (i.e. without neighbor sampling).
        g : the entire graph.
        x : the input of entire node set.
        The inference code is written in a fashion that it could handle any number of nodes and
        layers.
        """
        # During inference with sampling, multi-layer blocks are very inefficient because
        # lots of computations in the first few layers are repeated.
        # Therefore, we compute the representation of all nodes layer by layer.  The nodes
        # on each layer are of course splitted in batches.
        # TODO: can we standardize this?
        for l, layer in enumerate(self.layers):
            y = th.zeros(
                g.num_nodes(),
                self.n_hidden if l != len(self.layers) - 1 else self.n_classes,
            ).to(device)

            sampler = dgl.dataloading.MultiLayerFullNeighborSampler(1)
            dataloader = dgl.dataloading.DataLoader(
                g,
                th.arange(g.num_nodes()),
                sampler,
                batch_size=self.batch_size,
                shuffle=True,
                drop_last=False,
                num_workers=self.num_workers,
            )

            for input_nodes, output_nodes, blocks in tqdm.tqdm(dataloader):
                block = blocks[0].int().to(device)

                h = x[input_nodes]
                h_dst = h[: block.num_dst_nodes()]
                h = layer(block, (h, h_dst))
                if l != len(self.layers) - 1:
                    h = self.activation(h)
                    h = self.dropout(h)

                y[output_nodes] = h

            x = y
        return y

In [9]:
def compute_acc(pred, labels):
    """
    Compute the accuracy of prediction given the labels.
    """
    return (th.argmax(pred, dim=1) == labels).float().sum() / len(pred)


def evaluate(model, g, nfeat, labels, val_nid, test_nid, device):
    """
    Evaluate the model on the validation set specified by ``val_mask``.
    g : The entire graph.
    inputs : The features of all the nodes.
    labels : The labels of all the nodes.
    val_mask : A 0-1 mask indicating which nodes do we actually compute the accuracy for.
    device : The GPU device to evaluate on.
    """
    model.eval()
    with th.no_grad():
        pred = model.inference(g, nfeat, device)
    model.train()
    return (
        compute_acc(pred[val_nid], labels[val_nid]),
        compute_acc(pred[test_nid], labels[test_nid]),
        pred,
    )

def load_subtensor(nfeat, labels, seeds, input_nodes):
    """
    Extracts features and labels for a set of nodes.
    """
    batch_inputs = nfeat[input_nodes]
    batch_labels = labels[seeds]
    return batch_inputs, batch_labels

# Train

In [12]:
train_nid, val_nid, test_nid, in_feats, labels, n_classes, nfeat, g = data

# Create PyTorch DataLoader for constructing blocks
sampler = dgl.dataloading.MultiLayerNeighborSampler(
    [int(fanout) for fanout in FAN_OUT.split(",")]
)
dataloader = dgl.dataloading.DataLoader(
    g,
    train_nid,
    sampler,
    batch_size=BATCH_SIZE,
    shuffle=True,
    drop_last=False,
    num_workers=NUM_WORKERS,
)

In [15]:
model = SAGE(
    in_feats,
    n_classes,  # 47
    BATCH_SIZE,
    NUM_WORKERS
)
model = model.to(device)
loss_fcn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=WD)

In [16]:
avg = 0
iter_tput = []
best_eval_acc = 0
best_test_acc = 0
for epoch in range(NUM_EPOCHS):
    tic = time.time()

    # Loop over the dataloader to sample the computation dependency graph as a list of
    # blocks.
    for step, (input_nodes, seeds, blocks) in enumerate(dataloader):
        tic_step = time.time()

        # copy block to gpu
        blocks = [blk.int().to(device) for blk in blocks]

        # Load the input features as well as output labels
        batch_inputs, batch_labels = load_subtensor(
            nfeat, labels, seeds, input_nodes
        )

        # Compute loss and prediction
        batch_pred = model(blocks, batch_inputs)
        loss = loss_fcn(batch_pred, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        iter_tput.append(len(seeds) / (time.time() - tic_step))
        if step % LOG_EVERY == 0:
            acc = compute_acc(batch_pred, batch_labels)
            gpu_mem_alloc = (
                th.cuda.max_memory_allocated() / 1000000
                if th.cuda.is_available()
                else 0
            )
            print(
                "Epoch {:05d} | Step {:05d} | Loss {:.4f} | Train Acc {:.4f} | Speed (samples/sec) {:.4f} | GPU {:.1f} MB".format(
                    epoch,
                    step,
                    loss.item(),
                    acc.item(),
                    np.mean(iter_tput[3:]),
                    gpu_mem_alloc,
                )
            )
        gc.collect()

    toc = time.time()
    print("Epoch Time(s): {:.4f}".format(toc - tic))
    if epoch >= 5:
        avg += toc - tic
    if epoch % EVAL_EVERY == 0 and epoch != 0:
        eval_acc, test_acc, pred = evaluate(
            model, g, nfeat, labels, val_nid, test_nid, device
        )
        if SAVE_PRED:
            np.savetxt(
                SAVE_PRED + "%02d" % epoch,
                pred.argmax(1).cpu().numpy(),
                "%d",
            )
        print("Eval Acc {:.4f}".format(eval_acc))
        if eval_acc > best_eval_acc:
            best_eval_acc = eval_acc
            best_test_acc = test_acc
        print(
            "Best Eval Acc {:.4f} Test Acc {:.4f}".format(
                best_eval_acc, best_test_acc
            )
        )
    gc.collect()

print("Avg epoch time: {}".format(avg / (epoch - 4)))

/usr/local/lib/python3.10/dist-packages/dgl/dataloading/dataloader.py:1144: DGLWarning: Dataloader CPU affinity opt is not enabled, consider switching it on (see enable_cpu_affinity() or CPU best practices for DGL [https://docs.dgl.ai/tutorials/cpu/cpu_best_practises.html])
  dgl_warning(
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Epoch 00000 | Step 00000 | Loss 7.8669 | Train Acc 0.0160 | Speed (samples/sec) nan | GPU 1697.9 MB
Epoch 00000 | Step 00020 | Loss 1.2444 | Train Acc 0.6660 | Speed (samples/sec) 54648.9076 | GPU 1715.1 MB
Epoch 00000 | Step 00040 | Loss 0.8931 | Train Acc 0.7840 | Speed (samples/sec) 56410.1906 | GPU 1716.2 MB
Epoch 00000 | Step 00060 | Loss 0.7186 | Train Acc 0.8090 | Speed (samples/sec) 56845.8301 | GPU 1716.2 MB
Epoch 00000 | Step 00080 | Loss 0.7525 | Train Acc 0.8160 | Speed (samples/sec) 57523.4064 | GPU 1716.2 MB
Epoch 00000 | Step 00100 | Loss 0.6755 | Train Acc 0.8390 | Speed (samples/sec) 58215.0048 | GPU 1716.2 MB
Epoch 00000 | Step 00120 | Loss 0.4988 | Train Acc 0.8620 | Speed (samples/sec) 58889.3530 | GPU 1716.2 MB
Epoch 00000 | Step 00140 | Loss 0.5575 | Train Acc 0.8520 | Speed (samples/sec) 58809.2217 | GPU 1716.2 MB
Epoch 00000 | Step 00160 | Loss 0.5117 | Train Acc 0.8770 | Speed (samples/sec) 58599.1781 | GPU 1716.2 MB
Epoch 00000 | Step 00180 | Loss 0.4651 | Tra

100%|██████████| 2450/2450 [00:17<00:00, 139.32it/s]


Eval Acc 0.8967
Best Eval Acc 0.8967 Test Acc 0.7446
Epoch 00002 | Step 00000 | Loss 0.4104 | Train Acc 0.8770 | Speed (samples/sec) 57623.3602 | GPU 6350.8 MB
Epoch 00002 | Step 00020 | Loss 0.3954 | Train Acc 0.9030 | Speed (samples/sec) 57510.4399 | GPU 6350.8 MB
Epoch 00002 | Step 00040 | Loss 0.3700 | Train Acc 0.9010 | Speed (samples/sec) 57372.3442 | GPU 6350.8 MB
Epoch 00002 | Step 00060 | Loss 0.3717 | Train Acc 0.9060 | Speed (samples/sec) 57373.6411 | GPU 6350.8 MB
Epoch 00002 | Step 00080 | Loss 0.3743 | Train Acc 0.8940 | Speed (samples/sec) 57276.2813 | GPU 6350.8 MB
Epoch 00002 | Step 00100 | Loss 0.4032 | Train Acc 0.8900 | Speed (samples/sec) 57341.6840 | GPU 6350.8 MB
Epoch 00002 | Step 00120 | Loss 0.4065 | Train Acc 0.8940 | Speed (samples/sec) 57350.1721 | GPU 6350.8 MB
Epoch 00002 | Step 00140 | Loss 0.3312 | Train Acc 0.9050 | Speed (samples/sec) 57233.8404 | GPU 6350.8 MB
Epoch 00002 | Step 00160 | Loss 0.3121 | Train Acc 0.9080 | Speed (samples/sec) 57006.3270 

100%|██████████| 2450/2450 [00:17<00:00, 139.92it/s]


...

Eval Acc 0.9143
Best Eval Acc 0.9151 Test Acc 0.7803
Epoch 00008 | Step 00000 | Loss 0.3176 | Train Acc 0.9060 | Speed (samples/sec) 54266.8503 | GPU 6813.5 MB
Epoch 00008 | Step 00020 | Loss 0.2783 | Train Acc 0.9210 | Speed (samples/sec) 54247.5224 | GPU 6813.5 MB
Epoch 00008 | Step 00040 | Loss 0.2798 | Train Acc 0.9240 | Speed (samples/sec) 54261.7108 | GPU 6813.5 MB
Epoch 00008 | Step 00060 | Loss 0.3063 | Train Acc 0.9150 | Speed (samples/sec) 54244.6313 | GPU 6813.5 MB
Epoch 00008 | Step 00080 | Loss 0.2800 | Train Acc 0.9220 | Speed (samples/sec) 54227.9949 | GPU 6813.5 MB
Epoch 00008 | Step 00100 | Loss 0.3361 | Train Acc 0.9020 | Speed (samples/sec) 54213.2014 | GPU 6813.5 MB
Epoch 00008 | Step 00120 | Loss 0.3206 | Train Acc 0.9040 | Speed (samples/sec) 54193.4070 | GPU 6813.5 MB
Epoch 00008 | Step 00140 | Loss 0.2509 | Train Acc 0.9350 | Speed (samples/sec) 54138.6640 | GPU 6813.5 MB
Epoch 00008 | Step 00160 | Loss 0.2907 | Train Acc 0.9110 | Speed (samples/sec) 54092.3547 

100%|██████████| 2450/2450 [00:17<00:00, 139.40it/s]


Eval Acc 0.9173
Best Eval Acc 0.9173 Test Acc 0.7779
Epoch 00009 | Step 00000 | Loss 0.2672 | Train Acc 0.9200 | Speed (samples/sec) 54046.1353 | GPU 6813.5 MB
Epoch 00009 | Step 00020 | Loss 0.3120 | Train Acc 0.9190 | Speed (samples/sec) 54010.8009 | GPU 6813.5 MB
Epoch 00009 | Step 00040 | Loss 0.3155 | Train Acc 0.9150 | Speed (samples/sec) 53939.0448 | GPU 6813.5 MB
Epoch 00009 | Step 00060 | Loss 0.2699 | Train Acc 0.9190 | Speed (samples/sec) 53885.2760 | GPU 6813.5 MB
Epoch 00009 | Step 00080 | Loss 0.3167 | Train Acc 0.9030 | Speed (samples/sec) 53821.5605 | GPU 6813.5 MB
Epoch 00009 | Step 00100 | Loss 0.2884 | Train Acc 0.9210 | Speed (samples/sec) 53769.6091 | GPU 6813.5 MB
Epoch 00009 | Step 00120 | Loss 0.3383 | Train Acc 0.9120 | Speed (samples/sec) 53727.2403 | GPU 6813.5 MB
Epoch 00009 | Step 00140 | Loss 0.2700 | Train Acc 0.9250 | Speed (samples/sec) 53657.2414 | GPU 6813.5 MB
Epoch 00009 | Step 00160 | Loss 0.2587 | Train Acc 0.9230 | Speed (samples/sec) 53615.0276 

100%|██████████| 2450/2450 [00:17<00:00, 138.66it/s]

Eval Acc 0.9159
Best Eval Acc 0.9173 Test Acc 0.7779
Avg epoch time: 40.5114604473114


In [17]:
import torch
torch.save(model, './SAGE.pt')